<a href="https://colab.research.google.com/github/jgamel/learn_n_dev/blob/python_ds_examples/train_test_split_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of train_test_split()

### Step 1: Import packages and classes

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

Now that you have both imported, you can use them to split data into training sets and test sets. You’ll split inputs and outputs at the same time, with a single function call.

With train_test_split(), you need to provide the sequences that you want to split as well as any optional arguments. It returns a list of NumPy arrays, other sequences, or SciPy sparse matrices if appropriate:

arrays is the sequence of lists, NumPy arrays, pandas DataFrames, or similar array-like objects that hold the data you want to split. All these objects together make up the dataset and must be of the same length.

In supervised machine learning applications, you’ll typically work with two such sequences:

A two-dimensional array with the inputs (x)
A one-dimensional array with the outputs (y)
options are the optional keyword arguments that you can use to get desired behavior:

train_size is the number that defines the size of the training set. If you provide a float, then it must be between 0.0 and 1.0 and will define the share of the dataset used for testing. If you provide an int, then it will represent the total number of the training samples. The default value is None.

test_size is the number that defines the size of the test set. It’s very similar to train_size. You should provide either train_size or test_size. If neither is given, then the default share of the dataset that will be used for testing is 0.25, or 25 percent.

random_state is the object that controls randomization during splitting. It can be either an int or an instance of RandomState. The default value is None.

shuffle is the Boolean object (True by default) that determines whether to shuffle the dataset before applying the split.

stratify is an array-like object that, if not None, determines how to use a stratified split.

Now it’s time to try data splitting! You’ll start by creating a simple dataset to work with. The dataset will contain the inputs in the two-dimensional array x and outputs in the one-dimensional array y:

### Step 2: Get Data

In [4]:
x = np.arange(1, 25).reshape(12, 2)

y = np.array([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0])

array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12],
       [13, 14],
       [15, 16],
       [17, 18],
       [19, 20],
       [21, 22],
       [23, 24]])

In [5]:
x

array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12],
       [13, 14],
       [15, 16],
       [17, 18],
       [19, 20],
       [21, 22],
       [23, 24]])

In [6]:
y

array([0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0])

To get your data, you use arange(), which is very convenient for generating arrays based on numerical ranges. You also use .reshape() to modify the shape of the array returned by arange() and get a two-dimensional data structure.

You can split both input and output datasets with a single function call:

### Step 3: Split Data

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
x_train

array([[21, 22],
       [23, 24],
       [15, 16],
       [19, 20],
       [ 7,  8],
       [11, 12],
       [17, 18],
       [ 5,  6],
       [ 1,  2]])

In [18]:
print(x_test, end='\n\n')

print(y_train, end='\n\n')

print(y_test, end='\n\n')

[[13 14]
 [ 9 10]
 [ 3  4]]

[1 0 1 0 0 0 1 1 0]

[0 1 1]



Given two sequences, like x and y here, train_test_split() performs the split and returns four sequences (in this case NumPy arrays) in this order:

x_train: The training part of the first sequence (x)
x_test: The test part of the first sequence (x)
y_train: The training part of the second sequence (y)
y_test: The test part of the second sequence (y)
You probably got different results from what you see here. This is because dataset splitting is random by default. The result differs each time you run the function. However, this often isn’t what you want.

Sometimes, to make your tests reproducible, you need a random split with the same output for each function call. You can do that with the parameter random_state. The value of random_state isn’t important—it can be any non-negative integer. You could use an instance of numpy.random.RandomState instead, but that is a more complex approach.

In the previous example, you used a dataset with twelve observations (rows) and got a training sample with nine rows and a test sample with three rows. That’s because you didn’t specify the desired size of the training and test sets. By default, 25 percent of samples are assigned to the test set. This ratio is generally fine for many applications, but it’s not always what you need.

Typically, you’ll want to define the size of the test (or training) set explicitly, and sometimes you’ll even want to experiment with different values. You can do that with the parameters train_size or test_size.

Modify the code so you can choose the size of the test set and get a reproducible result:

In [20]:
x_train, x_test, y_train, y_test = train_test_split(
     x, y, test_size=4, random_state=4
 )
print (x_train, end='\n\n')

print (x_test, end='\n\n')

print (y_train, end='\n\n')

print (y_test, end='\n\n')

[[17 18]
 [ 5  6]
 [23 24]
 [ 1  2]
 [ 3  4]
 [11 12]
 [15 16]
 [21 22]]

[[ 7  8]
 [ 9 10]
 [13 14]
 [19 20]]

[1 1 0 0 1 0 1 1]

[0 1 0 0]



With this change, you get a different result from before. Earlier, you had a training set with nine items and test set with three items. Now, thanks to the argument test_size=4, the training set has eight items and the test set has four items. You’d get the same result with test_size=0.33 because 33 percent of twelve is approximately four.

There’s one more very important difference between the last two examples: You now get the same result each time you run the function. This is because you’ve fixed the random number generator with random_state=4.

### Stratified Sample

If you want to (approximately) keep the proportion of y values through the training and test sets, then pass stratify=y. This will enable stratified splitting:

In [21]:
x_train, x_test, y_train, y_test = train_test_split(
     x, y, test_size=0.33, random_state=4, stratify=y
 )


In [22]:
x_train

array([[21, 22],
       [ 1,  2],
       [15, 16],
       [13, 14],
       [17, 18],
       [19, 20],
       [23, 24],
       [ 3,  4]])

In [23]:
x_test

array([[11, 12],
       [ 7,  8],
       [ 5,  6],
       [ 9, 10]])

In [24]:
y_train

array([1, 0, 1, 0, 1, 0, 0, 1])

In [25]:
y_test

array([0, 0, 1, 1])

Now y_train and y_test have the same ratio of zeros and ones as the original y array.

Stratified splits are desirable in some cases, like when you’re classifying an imbalanced dataset, a dataset with a significant difference in the number of samples that belong to distinct classes.

Finally, you can turn off data shuffling and random split with shuffle=False:

In [26]:
x_train, x_test, y_train, y_test = train_test_split(
     x, y, test_size=0.33, shuffle=False
 )

In [27]:
x_train

array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10],
       [11, 12],
       [13, 14],
       [15, 16]])

In [28]:
x_test

array([[17, 18],
       [19, 20],
       [21, 22],
       [23, 24]])

In [29]:
y_train

array([0, 1, 1, 0, 1, 0, 0, 1])

In [30]:
y_test

array([1, 0, 1, 0])

Now you have a split in which the first two-thirds of samples in the original x and y arrays are assigned to the training set and the last third to the test set. No shuffling. No randomness.

# Supervised Machine Learning With train_test_split()

Now it’s time to see train_test_split() in action when solving supervised learning problems. You’ll start with a small regression problem that can be solved with linear regression before looking at a bigger problem. You’ll also see that you can use train_test_split() for classification as well.

Minimalist Example of Linear Regression
In this example, you’ll apply what you’ve learned so far to solve a small regression problem. You’ll learn how to create datasets, split them into training and test subsets, and use them for linear regression.

As always, you’ll start by importing the necessary packages, functions, or classes. You’ll need NumPy, LinearRegression, and train_test_split():

In [31]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Now that you’ve imported everything you need, you can create two small arrays, x and y, to represent the observations and then split them into training and test sets just as you did before:

In [33]:
x = np.arange(20).reshape(-1, 1)
y = np.array([5, 12, 11, 19, 30, 29, 23, 40, 51, 54, 74,
               62, 68, 73, 89, 84, 89, 101, 99, 106])
print(x, end='\n\n')

print(y, end='\n\n')



x_train, x_test, y_train, y_test = train_test_split(
     x, y, test_size=8, random_state=0
)

[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]]

[  5  12  11  19  30  29  23  40  51  54  74  62  68  73  89  84  89 101
  99 106]



Your dataset has twenty observations, or x-y pairs. You specify the argument test_size=8, so the dataset is divided into a training set with twelve observations and a test set with eight observations.

Now you can use the training set to fit the model:

In [34]:
model = LinearRegression().fit(x_train, y_train)
model.intercept_


3.1617195496417523

In [35]:
model.coef_

array([5.53121801])

LinearRegression creates the object that represents the model, while .fit() trains, or fits, the model and returns it. With linear regression, fitting the model means determining the best intercept (model.intercept_) and slope (model.coef_) values of the regression line.

Although you can use x_train and y_train to check the goodness of fit, this isn’t a best practice. An unbiased estimation of the predictive performance of your model is based on test data:

In [36]:
model.score(x_train, y_train)

0.9868175024574795

In [37]:
model.score(x_test, y_test)

0.9465896927715023

.score() returns the coefficient of determination, or R², for the data passed. Its maximum is 1. The higher the R² value, the better the fit. In this case, the training data yields a slightly higher coefficient. However, the R² calculated with test data is an unbiased measure of your model’s prediction performance.



## Regression Example

Now you’re ready to split a larger dataset to solve a regression problem. You’ll use a well-known [Boston house prices](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn.datasets.load_boston) dataset, which is included in sklearn. This dataset has 506 samples, 13 input variables, and the house values as the output. You can retrieve it with load_boston().

First, import train_test_split() and load_boston():


In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

Now that you have both functions imported, you can get the data to work with:

In [2]:
x, y = load_boston(return_X_y=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

As you can see, load_boston() with the argument return_X_y=True returns a tuple with two NumPy arrays:

A two-dimensional array with the inputs
A one-dimensional array with the outputs
The next step is to split the data the same way as before:

In [3]:
x_train, x_test, y_train, y_test = train_test_split(
     x, y, test_size=0.4, random_state=0
 )

Now you have the training and test sets. The training data is contained in x_train and y_train, while the data for testing is in x_test and y_test.

When you work with larger datasets, it’s usually more convenient to pass the training or test size as a ratio. test_size=0.4 means that approximately 40 percent of samples will be assigned to the test data, and the remaining 60 percent will be assigned to the training data.

Finally, you can use the training set (x_train and y_train) to fit the model and the test set (x_test and y_test) for an unbiased evaluation of the model. In this example, you’ll apply three well-known regression algorithms to create models that fit your data:

Linear regression with LinearRegression()

[Gradient boosting](https://en.wikipedia.org/wiki/Gradient_boosting) with [GradientBoostingRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)

Random forest with RandomForestRegressor()


The process is pretty much the same as with the previous example:

Import the classes you need.
Create model instances using these classes.
Fit the model instances with .fit() using the training set.
Evaluate the model with .score() using the test set.
Here’s the code that follows the steps described above for all three regression algorithms:

### Linear Regression

In [5]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(x_train, y_train)


In [6]:

model.score(x_train, y_train)

0.7668160223286261

In [7]:
model.score(x_test, y_test)

0.688260714253802

### Gradient Boosting

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=0).fit(x_train, y_train)

In [9]:
model.score(x_train, y_train)

0.9859065238883613

In [10]:
model.score(x_test, y_test)

0.8530127436482149

### Random Forest

In [11]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=0).fit(x_train, y_train)

In [12]:
model.score(x_train, y_train)

0.9811695664860354

In [13]:
model.score(x_test, y_test)

0.8325867908704008

You’ve used your training and test datasets to fit three models and evaluate their performance. The measure of accuracy obtained with .score() is the coefficient of determination. It can be calculated with either the training or test set. However, as you already learned, the score obtained with the test set represents an unbiased estimation of performance.

As mentioned in the documentation, you can provide optional arguments to LinearRegression(), GradientBoostingRegressor(), and RandomForestRegressor(). GradientBoostingRegressor() and RandomForestRegressor() use the random_state parameter for the same reason that train_test_split() does: to deal with randomness in the algorithms and ensure reproducibility.

## Classification Example

You can use train_test_split() to solve classification problems the same way you do for regression analysis. In machine learning, classification problems involve training a model to apply labels to, or classify, the input values and sort your dataset into categories.

In the tutorial [Logistic Regression](https://realpython.com/logistic-regression-python/) in Python, you’ll find an example of a [handwriting recognition](https://realpython.com/logistic-regression-python/#logistic-regression-in-python-handwriting-recognition) task. The example provides another demonstration of splitting data into training and test sets to avoid bias in the evaluation process.

### Other Validation Functionalities

The package sklearn.model_selection offers a lot of functionalities related to model selection and validation, including the following:

Cross-validation

Learning curves

Hyperparameter tuning

[Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) is a set of techniques that combine the measures of prediction performance to get more accurate model estimations.





One of the widely used cross-validation methods is [k-fold cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation). In it, you divide your dataset into k (often five or ten) subsets, or folds, of equal size and then perform the training and test procedures k times. Each time, you use a different fold as the test set and all the remaining folds as the training set. This provides k measures of predictive performance, and you can then analyze their mean and standard deviation.

You can implement cross-validation with [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html), [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html), [LeaveOneOut](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html), and a few other classes and functions from sklearn.model_selection.

A [learning curve](https://en.wikipedia.org/wiki/Learning_curve_(machine_learning)), sometimes called a training curve, shows how the prediction score of training and validation sets depends on the number of training samples. You can use [learning_curve()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html) to get this dependency, which can help you find the optimal size of the training set, choose hyperparameters, compare models, and so on.

[Hyperparameter tuning](https://en.wikipedia.org/wiki/Hyperparameter_optimization), also called hyperparameter optimization, is the process of determining the best set of hyperparameters to define your machine learning model. sklearn.model_selection provides you with several options for this purpose, including [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html), [validation_curve()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.validation_curve.html), and others. Splitting your data is also important for hyperparameter tuning.

Conclusion
You now know why and how to use train_test_split() from sklearn. You’ve learned that, for an unbiased estimation of the predictive performance of machine learning models, you should use data that hasn’t been used for model fitting. That’s why you need to split your dataset into training, test, and in some cases, validation subsets.

In this tutorial, you’ve learned how to:

Use train_test_split() to get training and test sets
Control the size of the subsets with the parameters train_size and test_size
Determine the randomness of your splits with the random_state parameter
Obtain stratified splits with the stratify parameter
Use train_test_split() as a part of supervised machine learning procedures
